In [1]:
### Combined metadata 
from Bio import SeqIO
import os,sys
import pandas as pd
import phylopandas as ph
import numpy as np

In [13]:
## add genotype ref
meta = pd.read_csv('HRSV_0622.csv')
## tab deleiminated text file 
genotype = pd.read_csv("Gbmunge/HRSV_0622_gbmeta.txt",sep='\t', lineterminator='\r')

meta2= pd.merge(meta, genotype, how='left', left_on='Accession', right_on='accession')

## add subtype infor
subtype = pd.read_csv("HRSV_subtype.txt",sep='\t')
subtype[['accession','version']] = subtype.Accession.str.split(".",expand=True)
subtype
meta3= pd.merge(meta2, subtype, how='left', left_on='Accession', right_on='accession')
meta3.to_csv("HRSV_0622_meta.csv", index = False)

In [19]:
## code the taxa with metadata

def taxa(meta_file,fasta_file,output):
    meta = pd.read_csv(meta_file)
    
    df = ph.read_fasta(fasta_file)
    df = df.filter(['id','sequence'], axis=1)
    df[['Accession','version']] = df.id.str.split(".",expand=True) 
    df = df.filter(['Accession','sequence'])
    
    fasta=pd.merge(df, meta, how='left', left_on='Accession', right_on='Accession_x')
    fasta['taxa']=fasta.apply(lambda x:'%s-%s-%s-%s-%s' % (x['Accession_x'],x['Subtype'],x['country_code'],x['Collection_Date_code'],x['genotype_ref']),axis=1)
    fasta=fasta.filter(['taxa','sequence'], axis=1)
    fasta.to_csv("temp.tab", sep="\t",index = False,header=False)
    fasta = SeqIO.parse("temp.tab", "tab")
    SeqIO.write(fasta, output, "fasta")
    os.remove("temp.tab")
    print("All taxa in "+ fasta_file + " have been renamed!")

In [20]:
taxa('HRSV_0622_meta_code_0627.csv','clear_rmHRSV_F.fasta','HRSV_F_taxa_0624.fasta')

All taxa in clear_rmHRSV_F.fasta have been renamed!


In [13]:
## remove artifical sequence with shell script
##`sh rm_taxa.sh HRSV_F_taxa_0624.fasta HRSV_artificial_acession.txt > HRSV_F_0627clean.fasta`

In [21]:
## seperate sequence based on the avaliable subtype information
def subsetAB(fasta,outputA,outputB):
    df = ph.read_fasta(fasta)
    df = df.filter(['id','sequence'], axis=1)
    df[['Accession','Subtype','country_code','collection_date','genotype_ref']] = df.id.str.split("-",expand=True)
    check =df[pd.isnull(df['Subtype'])]
    if check.empty == True:
        print ("Sequence are seperate into subtype A and B !!!")
        RSVA = df[df['Subtype'] =='A']
        RSVA = RSVA.filter(['id','sequence'], axis=1)
        RSVA.to_csv("temp.tab", sep="\t",index = False,header=False)
        RSVA = SeqIO.parse("temp.tab", "tab")
        SeqIO.write(RSVA, outputA, "fasta")
    
        RSVB = df[df['Subtype'] =='B']
        RSVB = RSVB.filter(['id','sequence'], axis=1)
        RSVB.to_csv("temp.tab", sep="\t",index = False,header=False)
        RSVB = SeqIO.parse("temp.tab", "tab")
        SeqIO.write(RSVB, outputB, "fasta")
        os.remove("temp.tab")
    
    else:
        print("Sequence without subtype has been found!!!")
        print(check)

In [23]:
subsetAB("HRSV_F_clean0627.fasta","HRSVA_F_clean0627.fasta","HRSVB_F_clean_0627.fasta")

Sequence are seperate into subtype A and B !!!
